In [30]:
import pandas as pd
import urllib.request
import numpy as np
import requests
import unicodedata
from PIL import Image
import time
import datetime

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

def sanitizar_nombre(name):
    return strip_accents((name.strip().replace(" ", "_").replace("(","").replace(")","")).lower()).replace('&#34;','')

def get_name(x):
    sanitized = sanitizar_nombre(x[2])
    return sanitized[:35]

# RECIBE FILA DEL DATAFRAME
# precondición: el campo imagen no es NaN
def obtener_url_img(y):
    url = y[7].split('":"')[1]
    url2 = url.split(",")[0][:-1]
    return url2

# RECIBE FILA DEL DATAFRAME
def obtenerlink(y):
    #print(y)
    url = y[5].split('":"')[1]
    url2 = url2 = url.split(",")[0][:-1]
    url2 = url2.replace('"','')
    return url2

def quote(str):
    return '"'+str+'"'

In [2]:
# Identify type of timeline event (general information (logo_info) vs tourist information (logo_tour))
def find_type(event):
    if(not pd.isnull(event[7])):
        #not a NaN
        if(logo_info in event[7]):
            return "informacion"
        elif(logo_tour in event[7]):
            return "turismo"
        else:
            raise Exception("Image unrecognized: "+event[7]) 
    else:
        return 0

In [59]:
def crear_evento(x):
    idn = get_name(x)
    #print(idn)
    #reemplazar NaNs en image por 0!!
    path_img = quote("0")
    width = 0
    start = quote(x[0][:-9])
    # Convert bce dates to a format interpreted by vis.js
    if("bce" in x[0][:-9]):
        date=x[0][:-9].split("-")
        start = "new Date(-"+ date[1] +","+ date[2] +","+ date[3] +")"
    end = "0"
    if(x[0][:-9] != x[1][:-9]):
        end = quote(x[1][:-9])
    
    name_event = x[2].replace("&#34;", '\\"')
    count_links = 0
    
    
    if(check_repeats.count(idn)==0):
        #Event is not repeated
        if(not pd.isnull(x[5])):
            count_links = 1
            links = "["+ quote(obtenerlink(x)) + "]" 
    else:
        if(handled_repeats.count(idn)!=0):
            raise Exception("Event already created") 
        handled_repeats.append(idn) #we only want one event with all the data
        if(not pd.isnull(x[5])):
            count_links = 2 #more than 1 link = more than 1 work on same event
            filtro = df['title'].str.contains(x[2][:21])
            df_links = df[filtro]
            links = "["
            df_links = df_links.apply(obtenerlink,axis=1)
            for item in df_links.iteritems():
                url = item[1]
                links += quote(url) + ","
            links = links[:-1] + "]" #saco la , al pedo y agrego el corchete final
    
    if(pd.isnull(x[5])):
        links = "["+"]"
    
    img = find_type(x)
    content  = quote(name_event) 
    
    if(img!= 0):
        #if there is an image, include image
        path_img = quote('../img/'+img+'.svg')
        content += ", image:"+ path_img 
        
    if(count_links>1):
        #if there is multiple links, pass list of links
        content += ", links:" + links
    elif(count_links==1):
        #if there is only one link, pass single link
        content += ", link:" + links[1:][:-1]
    
    res = "{id: " + quote(idn) + ", content: " + content + ", start: " + start 
    if(end!="0"):
        res += ", end: " + end
        
    res += "}"
    return res

def try_create(x):
    try:
        return crear_evento(x)
        # If exception is raised, no return
    except Exception as exc:
        print('[!!!] Evento repetido:',x[2])

print("event:",crear_evento(df.iloc[1]))
print(handled_repeats)

event: {id: "9000_ac:_revolucion_neolitica_gordo", content: "9000 aC: Revolución Neolítica (Gordon Childe)", image:"../img/informacion.svg", link:"https://drive.google.com/file/d/1RnP1ORXvdTbsCBDmryAYdafjXOFPgi1y/view?usp=sharing", start: new Date(-9000,01,01)}
['476_fin_imperio_romano_y_edad_antig']


In [62]:
unit_name="1unidad"
df = pd.read_csv("./"+unit_name+"/"+unit_name+".csv",index_col=False)
df.columns = df.columns.str.replace(' ', '')
logo_info = "www.cooperacion2005.es"
logo_tour = "guejarsierra.org"

names = df.apply(get_name,axis=1)
seen = set()
check_repeats = []
for x in names:
    if x not in seen:
        seen.add(x)
    else:
        check_repeats.append(x)
handled_repeats=[]
print(check_repeats)

def validate_date(date,start_date):
    for i in range(3):
        start_date[i] = int(start_date[i])
        date[i] = int(date[i])

    if((start_date[2]<date[2]) or 
        (start_date[2]==date[2] and start_date[1]<date[1]) or 
        (start_date[2]==date[2] and start_date[1]==date[1] and start_date[0]<=date[0])):
        return True
    else:
        return False

def sorter(x):
    splt = x.split('start: ')[1].replace('"','').replace("'","").replace("}","")
    date = splt.split('-')
    if("Date" in date[0]):
        return "1unidad"
    if(int(date[0])<1810):
        return "1unidad"
    if(int(date[0])>=1810 and int(date[0])<1914):
        return "2unidad"
    if(int(date[0])>=1914 and int(date[0])<1929):
        return "3unidad"
    else:
        return "otro"

start_line = "var items = new vis.DataSet(["
end_line = "]);"

handled_repeats = []

print("Procesando base de datos...")
df_copy = df.copy()
events = df_copy.apply(try_create,axis=1)
events = events.dropna()
unidad = events.copy().apply(sorter)
frame = { 'Event': events, 'Unit': unidad } 
cosas = pd.DataFrame(frame) 

unit1 = cosas[cosas['Unit']=='1unidad']
unit2 = cosas[cosas['Unit']=='2unidad']
#unit3 = cosas[cosas['Unit']=='3unidad']

#del unit1["Unit"]
#display(unit1.head())

#raise Exception
#units = [unit1,unit2,unit3]
#unit_names = ["1unidad","2unidad","3unidad"]
units = [unit1,unit2]
unit_names = ["1unidad","2unidad"]

for i in range(2):
    print("Guardando en archivo...")
    unit_name = unit_names[i]
    unit = units[i]
    del unit["Unit"]
    f = open("./"+unit_name+"/"+unit_name+"_appendable.js", "w")
    f.write(start_line)
    f.write("\n")

    for _,linea in unit[:-1].iterrows():
        #lleno con todos los eventos hasta el anteultimo
        f.write(linea["Event"] + ",\n")

    #La ULTIMA fila debe ir SIN COMA en la base finalizada, y CON COMA en la appendable!
    f.close()
    print("Diferenciando versiones appendable y finalizada")
    !cp "./{unit_name}/{unit_name}_appendable.js" "./{unit_name}/{unit_name}.js"

    f2 = open("./"+unit_name+"/"+unit_name+".js","a")
    #aca va sin coma y con linea finalizadora
    f2.write(unit.iloc[-1]["Event"]+"\n")
    f2.write(end_line)
    f2.close()
    print("Version finalizada creada")

    #La version appendable va con coma y sin linea finalizadora
    f = open("./"+unit_name+"/"+unit_name+"_appendable.js", "a")
    f.write(cosas.iloc[-1]["Event"]+",")
    f.close()
    print("Versión appendable creada")
    print("Finalizado!")

['476_fin_imperio_romano_y_edad_antig', '476_fin_imperio_romano_y_edad_antig', '476_fin_imperio_romano_y_edad_antig', '476_fin_imperio_romano_y_edad_antig']
Procesando base de datos...
[!!!] Evento repetido: 476 Fin Imperio Romano y Edad Antigua (2)
[!!!] Evento repetido: 476 Fin Imperio Romano y Edad Antigua (3)
[!!!] Evento repetido: 476 Fin Imperio Romano y Edad Antigua (4)
[!!!] Evento repetido: 476 Fin Imperio Romano y Edad Antigua (5)
Guardando en archivo...
Diferenciando versiones appendable y finalizada
Version finalizada creada
Versión appendable creada
Finalizado!
Guardando en archivo...
Diferenciando versiones appendable y finalizada
Version finalizada creada
Versión appendable creada
Finalizado!
